In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier, XGBRegressor
from sklearn.metrics import accuracy_score
import ta
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
# Data Load
df = pd.read_csv('D:/XGBoost/Samsung.txt', sep = ',')

In [3]:
df

,MARKET,NAME,SYMBOL,DATEDEAL,OPEN,HIGH,LOW,CLOSE,DIFF,VOLUME
0,KOSPI,삼성전자,5930,2016-01-04,25200,25200,24100,24100,-4.37,306939
1,KOSPI,삼성전자,5930,2016-01-05,24040,24360,23720,24160,0.25,216002
2,KOSPI,삼성전자,5930,2016-01-06,24160,24160,23360,23500,-2.73,366752
3,KOSPI,삼성전자,5930,2016-01-07,23320,23660,23020,23260,-1.02,282388
4,KOSPI,삼성전자,5930,2016-01-08,23260,23720,23260,23420,0.69,257763
...,...,...,...,...,...,...,...,...,...,...
1470,KOSPI,삼성전자,5930,2021-12-24,80200,80800,80200,80500,0.75,12086380
1471,KOSPI,삼성전자,5930,2021-12-27,80600,80600,79800,80200,-0.37,10783368
1472,KOSPI,삼성전자,5930,2021-12-28,80200,80400,79700,80300,0.12,18226325
1473,KOSPI,삼성전자,5930,2021-12-29,80200,80200,78500,78800,-1.87,19794795


In [4]:
# Make Indicator : SMA, EMA, Stochastic K%, Stochastic D%, RSI, MACD, Disparity

# Bollinger Band
bol_h = ta.volatility.bollinger_hband(df["CLOSE"])
bol_l = ta.volatility.bollinger_lband(df["CLOSE"])

# SMA
sma = ta.trend.sma_indicator(df["CLOSE"],12)

# EMA
ema = ta.trend.ema_indicator(df["CLOSE"],12)

# Stochastic
sto_k = ta.momentum.stochrsi_k(df["CLOSE"])
sto_d = ta.momentum.stochrsi_d(df["CLOSE"])

# RSI
rsi = ta.momentum.rsi(df["CLOSE"], 14)

# MACD
macd = ta.trend.macd(df["CLOSE"], 13, 26)

In [6]:
updown_list = []
for i in range(len(df)):
    if i == 0:
        print("야옹")
    else:
        updown = df["CLOSE"][i] - df["CLOSE"][i-1]
        if updown >= 0:
            updown_list.append(1)
        else:
            updown_list.append(0)
print(len(updown_list))

야옹
1474


In [346]:
'''
# Concat Indicators and Price
x = np.zeros((len(df)-200-1,9))
idx = 0
for i in range(200,len(df)-1):
    x[idx][0] = bol_h[i]
    x[idx][1] = bol_l[i]
    x[idx][2] = sma[i]
    x[idx][3] = ema[i]
    x[idx][4] = sto_k[i]
    x[idx][5] = sto_d[i]
    x[idx][6] = rsi[i]
    x[idx][7] = macd[i]
    #x[idx][8] = df["CLOSE"][i]
    x[idx][8] = df["DIFF"][i]
    
    idx += 1
'''
# Concat Indicators and Price
x = np.zeros((len(df)-200-1,7))
idx = 0
for i in range(200,len(df)-1):
    x[idx][0] = sma[i]
    x[idx][1] = ema[i]
    x[idx][2] = sto_k[i]
    x[idx][3] = sto_d[i]
    x[idx][4] = rsi[i]
    x[idx][5] = macd[i]
    x[idx][6] = df["DIFF"][i]
    
    idx += 1
'''
# Concat Indicators and Price
x = np.zeros((len(df)-200-1,1))
idx = 0
for i in range(200,len(df)-1):
    x[idx][0] = df["DIFF"][i]
    
    idx += 1

# Up Down Labels
y = np.zeros((len(df)-200-1,1))
idx2 = 0
for j in range(200,len(df)-1):
    y[idx2][0] = updown_list[j]
    idx2 += 1
'''
# Next Day Close Price
y = np.zeros((len(df)-200-1,1))

idx2 = 0
for j in range(200,len(df)-1):
    y[idx2][0] = df["DIFF"][j+1]
    idx2 += 1

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle = True, random_state = 34)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, shuffle = True, random_state = 34)


In [347]:
X = []
Y = []
size = 10
idx = 0
for i in range(0, len(x)-size+1):
    x_1 = []
    y_1 = []
    for j in range(size):
        x_1.append(x[i+j][0])
        x_1.append(x[i+j][1])
        x_1.append(x[i+j][2])
        x_1.append(x[i+j][3])
        x_1.append(x[i+j][4])
        x_1.append(x[i+j][5])
        x_1.append(x[i+j][6])
        #x_1.append(x[i+j][7])
        #x_1.append(x[i+j][8])
    
    y_1.append(y[i+(size-1)][0])
    X.append(x_1)
    Y.append(y_1)
    
print(len(X))
print(len(X[0]))

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=123)

1265
70


In [355]:
# Model Build
#model = XGBClassifier(max_depth = 5,  n_estimators=300, learning_rate=0.05)
model = XGBRegressor(max_depth =5, n_estimators = 300, learning_rate = 0.0001)
#model = XGBRegressor()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

same_count = 0
for i in range(len(y_pred)):
    if y_pred[i] >= 0 and y_test[i][0] >= 0:
        same_count += 1
    elif y_pred[i] < 0 and y_test[i][0] < 0:
        same_count += 1
print(same_count / len(y_pred))

0.616600790513834


In [356]:
y_pred

array([0.48733708, 0.48733708, 0.48733708, 0.48733708, 0.48733708,
       0.48733708, 0.48733708, 0.48733708, 0.48733708, 0.48733708,
       0.48733708, 0.48733708, 0.48733708, 0.48733708, 0.48733708,
       0.48733708, 0.396208  , 0.48733708, 0.48733708, 0.48733708,
       0.48733708, 0.48733708, 0.48733708, 0.48733708, 0.48733708,
       0.48733708, 0.48733708, 0.48733708, 0.48733708, 0.48733708,
       0.48733708, 0.48733708, 0.48733708, 0.48733708, 0.48733708,
       0.48733708, 0.48733708, 0.48733708, 0.48733708, 0.48733708,
       0.48733708, 0.48733708, 0.48733708, 0.48733708, 0.48733708,
       0.48733708, 0.48733708, 0.48733708, 0.48733708, 0.48733708,
       0.48733708, 0.48733708, 0.48733708, 0.48733708, 0.48733708,
       0.48733708, 0.48733708, 0.48733708, 0.48733708, 0.48733708,
       0.48733708, 0.48733708, 0.48733708, 0.48733708, 0.48733708,
       0.48733708, 0.48733708, 0.48733708, 0.48733708, 0.44228798,
       0.48733708, 0.48733708, 0.48733708, 0.48733708, 0.48733

In [345]:
y_test

[[0.67],
 [0.29],
 [-2.22],
 [-0.84],
 [-1.16],
 [0.3],
 [1.08],
 [0.14],
 [0.22],
 [0.74],
 [1.84],
 [3.24],
 [1.41],
 [-1.0],
 [-0.55],
 [1.0],
 [2.74],
 [-1.31],
 [1.95],
 [1.16],
 [0.22],
 [0.37],
 [2.76],
 [2.29],
 [0.12],
 [0.69],
 [3.01],
 [0.0],
 [0.39],
 [-2.56],
 [-0.8],
 [0.43],
 [2.21],
 [2.79],
 [0.0],
 [0.8],
 [-0.07],
 [0.47],
 [3.17],
 [1.56],
 [-1.6],
 [0.92],
 [1.06],
 [0.24],
 [0.05],
 [0.13],
 [-2.29],
 [0.04],
 [0.0],
 [-1.37],
 [-0.1],
 [0.9],
 [0.78],
 [-1.08],
 [0.23],
 [0.57],
 [-1.71],
 [-3.53],
 [-1.86],
 [-3.45],
 [1.0],
 [-0.68],
 [-1.04],
 [-2.83],
 [0.69],
 [-0.52],
 [-2.25],
 [-0.99],
 [1.79],
 [0.19],
 [-2.41],
 [-0.21],
 [-0.15],
 [0.12],
 [-1.42],
 [1.73],
 [0.0],
 [2.53],
 [-0.42],
 [0.64],
 [-1.51],
 [0.85],
 [2.18],
 [-0.65],
 [-0.34],
 [0.04],
 [0.73],
 [0.7],
 [1.67],
 [-1.88],
 [-1.24],
 [-1.19],
 [0.0],
 [-4.1],
 [0.84],
 [-3.98],
 [0.91],
 [4.33],
 [0.33],
 [0.1],
 [1.63],
 [0.29],
 [1.09],
 [-1.7],
 [-2.58],
 [0.0],
 [-1.15],
 [0.5],
 [-3.2],

In [344]:
y_pred

array([0.49205142, 0.48395595, 0.48395595, 0.48395595, 0.48395595,
       0.49205142, 0.49205142, 0.48395595, 0.49205142, 0.48671132,
       0.49205142, 0.49205142, 0.48395595, 0.49205142, 0.48395595,
       0.49205142, 0.44391382, 0.48671132, 0.48395595, 0.49205142,
       0.48395595, 0.48671132, 0.48395595, 0.48395595, 0.49205142,
       0.48395595, 0.49205142, 0.49205142, 0.48395595, 0.48395595,
       0.48395595, 0.48671132, 0.48395595, 0.48671132, 0.48395595,
       0.48395595, 0.49205142, 0.48671132, 0.49205142, 0.48395595,
       0.49205142, 0.48395595, 0.48671132, 0.49205142, 0.49205142,
       0.48395595, 0.48395595, 0.48395595, 0.48671132, 0.49205142,
       0.48395595, 0.48395595, 0.48395595, 0.49205142, 0.49205142,
       0.48671132, 0.49205142, 0.48671132, 0.49205142, 0.49205142,
       0.49205142, 0.48671132, 0.49205142, 0.49205142, 0.48395595,
       0.49205142, 0.49205142, 0.49205142, 0.49205142, 0.47990134,
       0.49205142, 0.49205142, 0.48395595, 0.48395595, 0.48671

In [325]:
for ii in range(5,31):
    X = []
    Y = []
    size = ii
    idx = 0
    for i in range(0, len(x)-size+1):
        x_1 = []
        y_1 = []
        for j in range(size):
            x_1.append(x[i+j][0])
            #x_1.append(x[i+j][1])
            #x_1.append(x[i+j][2])
            #x_1.append(x[i+j][3])
            #x_1.append(x[i+j][4])
            #x_1.append(x[i+j][5])
            #x_1.append(x[i+j][6])
            #x_1.append(x[i+j][7])
            #x_1.append(x[i+j][8])
        
        y_1.append(y[i+(size-1)][0])
        X.append(x_1)
        Y.append(y_1)
        
    #print(len(X))
    #print(len(X[0]))

    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=123)

    # Model Build
    model = XGBRegressor(max_depth =10, n_estimators = 300, learning_rate = 0.000005)
    #model = XGBRegressor()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    same_count = 0
    for i in range(len(y_pred)):
        if y_pred[i] >= 0 and y_test[i][0] >= 0:
            same_count += 1
        elif y_pred[i] < 0 and y_test[i][0] < 0:
            same_count += 1
    print("size : {}, acc : {} ".format(ii,round(same_count / len(y_pred),2)))

size : 5, acc : 0.53 
size : 6, acc : 0.49 
size : 7, acc : 0.59 
size : 8, acc : 0.57 
size : 9, acc : 0.59 
size : 10, acc : 0.62 
size : 11, acc : 0.58 
size : 12, acc : 0.53 
size : 13, acc : 0.5 
size : 14, acc : 0.47 
size : 15, acc : 0.48 
size : 16, acc : 0.52 
size : 17, acc : 0.52 
size : 18, acc : 0.58 
size : 19, acc : 0.58 
size : 20, acc : 0.53 
size : 21, acc : 0.52 
size : 22, acc : 0.59 
size : 23, acc : 0.56 
size : 24, acc : 0.52 
size : 25, acc : 0.58 


KeyboardInterrupt: 

In [141]:
y_updown = []
y_updown2 = []
same_count = 0
for i in range(len(y_test)):
    if i >0:
        '''
        y_1 = y_test[i] - y_test[i-1]
        y_2 = y_pred[i] = y_pred[i-1]
        
        if y_1 >= 0 and y_2 >0:
            same_count += 1
        elif y_1 < 0 and y_2 < 0:
            same_count += 1
        '''
        if y_test[i] >= 0 and y_pred[i] >= 0:
            same_count += 1
        elif y_test[i] < 0 and y_pred[i] < 0:
            same_count += 1
print(same_count/(len(y_test)-1))

TypeError: '>=' not supported between instances of 'list' and 'int'

In [190]:
y_pred = model.predict(X_test)
# Check Accuracy
#acc = accuracy_score(y_test, predictions)
acc = accuracy_score(y_test, y_pred)
print("Accuracy : %.2f%%" % (acc*100.0))

Accuracy : 51.87%


In [138]:
# Predict
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

TypeError: type numpy.ndarray doesn't define __round__ method

In [147]:
# Check Precision
pre_1_list = [] # If predict value is 1 and real value is 1 then append 1
pre_0_list = [] # If Predict value is 0 and real value is 0 then append 1
for i in range(len(predictions)):
    if predictions[i] == 1:
        if y_test[i] == 1:
            pre_1_list.append(1)
        else:
            pre_1_list.append(0)
    elif predictions[i] == 0:
        if y_test[i] == 0:
            pre_0_list.append(1)
        else:
            pre_0_list.append(0)
pre_1 = round(sum(pre_1_list)/len(pre_1_list)*100,2)
pre_0 = round(sum(pre_0_list)/len(pre_0_list)*100,2)
print("Precision 1 : " + str(pre_1)+"%")
print("Precision 0 : " + str(pre_0)+"%")

IndexError: list index out of range

In [17]:
# Check Recall
up_count = 0
for i in range(len(updown_list)):
    if updown_list[i] == 1:
        up_count +=1
recall = round(up_count / len(updown_list)*100,2)
print("Recall : " + str(recall)+"%")

Recall : 55.29%


In [19]:
# And Compare to Just Close Price
x2 = np.zeros((len(df)-200,1))
idx = 0
for i in range(200,len(df)-1):
    x[idx][0] = df["CLOSE"][i]
    idx += 1

X_train, X_test, y_train, y_test = train_test_split(x2, y, test_size=0.2, random_state=123)

In [20]:
# Model Build2
model = XGBClassifier(max_depth = 5,  n_estimators=300, learning_rate=0.05)
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.05, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=5, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=300,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [21]:
# Predict2
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

In [22]:
# Check Accuracy2
acc = accuracy_score(y_test, predictions)
print("Accuracy : %.2f%%" % (acc*100.0))

Accuracy : 57.25%
